In [60]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from feature_engine.selection import DropConstantFeatures, DropDuplicateFeatures, SmartCorrelatedSelection
from feature_engine.encoding import OneHotEncoder
from feature_engine.selection import DropFeatures, DropConstantFeatures, DropDuplicateFeatures, SmartCorrelatedSelection
from feature_engine.imputation import AddMissingIndicator
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import MinMaxScaler
from feature_engine.discretisation import DecisionTreeDiscretiser
from sklearn.model_selection import GroupShuffleSplit

from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.imputation import ArbitraryNumberImputer
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import RareLabelEncoder

In [2]:
pd.set_option('display.max_columns', None)

In [7]:
df_all = pd.read_csv('/gpfs/milgram/project/rtaylor/imc33/data/clean/master_los.csv')

In [8]:
df_all.head()

,pat_enc_csn_id,admissiondate,dischargedate,pat_mrn_id,viz_primary_payer_name,viz_disp_collapsed,viz_age,viz_ethnicity,viz_race,viz_race_collapsed,viz_language,viz_insurance,viz_service_collapsed,viz_ynhhs_sg2_service,viz_observed_mortality_yn,viz_drg,viz_admission_day,viz_discharged_day,viz_outcome_prolonged_los_yn,thro_door_to_admit_order_hrs,thro_admit_order_to_first_bed_hrs,thro_last_bed_to_ed_dept_hrs,thro_admit_order_to_ed_dept_hrs,thro_boarding_yn,thro_ed_arrival_time,thro_ed_arrival_day,thro_admit_or_obs_order_time,thro_admit_or_obs_order_day,thro_first_bed_assigned_time,thro_first_bed_assigned_day,thro_last_bed_assigned_time,thro_last_bed_assigned_day,thro_ed_departure_time,thro_ed_departure_day,summary_consult_count_all,summary_consult_count_unique_services,summary_pt_consult_order_yn,summary_pt_consult_order_time,summary_pt_consult_order_day,summary_sw_consult_order_yn,summary_sw_consult_order_time,summary_sw_consult_order_day,summary_first_edd_time,summary_first_edd_day,summary_first_edd_doc_time,summary_first_edd_doc_day,summary_first_edd_to_discharge_hrs,summary_first_edd_doc_to_discharge_hrs,summary_last_edd_time,summary_last_edd_day,summary_last_edd_doc_time,summary_last_edd_doc_day,summary_last_edd_to_discharge_hrs,summary_last_edd_doc_to_discharge_hrs,summary_first_rfd_status,summary_first_rfd_time,summary_first_rfd_day,summary_first_rfd_to_discharge_hrs,summary_last_rfd_status,summary_last_rfd_time,summary_last_rfd_day,summary_last_rfd_to_discharge_hrs,summary_total_hrs_nmr,summary_total_hrs_mrwb,summary_total_hrs_mrnb,rfd_first_nmr_start_day,rfd_first_nmr_start_time,rfd_first_mrnb_start_day,rfd_first_mrnb_start_time,rfd_first_mrwb_start_day,rfd_first_mrwb_start_time,rfd_first_nmr_to_discharge_hrs,rfd_first_mrnb_to_discharge_hrs,rfd_first_mrwb_to_discharge_hrs,rfd_last_nmr_effective_end_day,rfd_last_nmr_effective_end_time,rfd_last_mrnb_effective_end_day,rfd_last_mrnb_effective_end_time,rfd_last_mrwb_effective_end_day,rfd_last_mrwb_effective_end_time,rfd_last_nmr_to_discharge_hrs,rfd_last_mrnb_to_discharge_hrs,rfd_last_mrwb_to_discharge_hrs,con_max_consult_order_to_sign_any_hrs,con_avg_consult_order_to_sign_any_hrs,con_max_consult_note_to_sign_any_hrs,con_avg_consult_note_to_sign_any_hrs,con_signer_ym_provider_count,con_signer_nemg_provider_count,con_signer_community_provider_count,con_max_consult_order_to_sign_colon_and_rectal_hrs,con_max_consult_order_to_sign_diabetes_hrs,con_max_consult_order_to_sign_hematology_hrs,con_max_consult_order_to_sign_infectious_disease_hrs,con_max_consult_order_to_sign_lab_medicine_hrs,con_max_consult_order_to_sign_nephrology_hrs,con_max_consult_order_to_sign_oncology_hrs,con_max_consult_order_to_sign_picc_hrs,con_max_consult_order_to_sign_palliative_hrs,con_max_consult_order_to_sign_pulmonology_hrs,con_max_consult_order_to_sign_radiation_oncology_hrs,con_max_consult_order_to_sign_radiology_hrs,con_max_consult_order_to_sign_urology_hrs,con_max_consult_order_to_sign_cardiology_hrs,con_max_consult_order_to_sign_gastroenterology_hrs,con_max_consult_order_to_sign_hepatology_hrs,con_max_consult_order_to_sign_other_hrs,con_max_consult_order_to_sign_pharmacy_hrs,con_max_consult_order_to_sign_plastic_surgery_hrs,con_max_consult_order_to_sign_surgery_hrs,con_max_consult_order_to_sign_vascular_surgery_hrs,con_max_consult_order_to_sign_neurology_hrs,con_max_consult_order_to_sign_hospitalist_service_hrs,con_max_consult_order_to_sign_psychiatry_hrs,con_max_consult_order_to_sign_internal_medicine_hrs,con_max_consult_order_to_sign_neuro_oncology_hrs,con_max_consult_order_to_sign_neurosurgery_hrs,con_max_consult_order_to_sign_ophthalmology_hrs,con_max_consult_order_to_sign_orthopedics_hrs,con_max_consult_order_to_sign_physical_medicine_and_rehabilitation_hrs,con_max_consult_order_to_sign_podiatry_hrs,con_max_consult_order_to_sign_dermatology_hrs,con_max_consult_order_to_sign_geriatrics_hrs,con_max_consult_order_to_sign_addiction_medicine_hrs,con_max_consult_order_to_sign_otolaryngology_ent_hrs,

In [9]:
df_all.shape

(13205, 337)

In [10]:
#Filter for only home patients
df_all["viz_disp_collapsed"].value_counts()

viz_disp_collapsed
Other    8346
Home     4859
Name: count, dtype: int64

In [11]:
df_home = df_all[df_all["viz_disp_collapsed"]=="Home"].copy()

In [14]:
df_home.shape

(4859, 337)

In [15]:
features_los = pd.read_excel('/gpfs/milgram/project/rtaylor/imc33/data/clean/features_los.xlsx')

In [16]:
features_los.head()

,col_name,type
0,pat_enc_csn_id,unchanged
1,pat_mrn_id,unchanged
2,viz_primary_payer_name,category
3,viz_disp_collapsed,category
4,viz_age,continuous


In [25]:
# Features that are divided into 5 types to conduct data preprocessing
# 1. drop: features that need to be dropped
drop_list = features_los[features_los['type'] == 'drop']['col_name'].tolist()

# 2. category: features that are already categorical and need to be OneHotEncoding (add missing value indicator, impute missing by adding 'missing' category)
category_list = features_los[features_los['type'] == 'category']['col_name'].tolist()

# 3. binary: features that are binary and need to be converted to categorical (add missing value indicator / prefer: fill missing with 0)
binary_list = features_los[features_los['type'] == 'binary']['col_name'].tolist()

# 4. continuous: features that are continous/numerical variables, need outlier handling and normalization (add missing value indicator, fill missing with zero, assuming absence of value means no imaging done except for age)
continuous_list = features_los[features_los['type'] == 'continuous']['col_name'].tolist()
#age isn't missing
#continuous_list.pop("age")

# 5. discrete: features that are discrete/numerical variables, need to be discretized (fill missing with 0)
discrete_list = features_los[features_los['type'] == 'discrete']['col_name'].tolist()

In [20]:
drop_list

['admissiondate', 'ed_arrival_time', 'ed_arrival_hour', 'ed_arrival_date']

In [22]:
#category_list

In [30]:
# Split into X, y
X = df_home.drop(['viz_outcome_prolonged_los_yn'], axis=1)
y = df_home['viz_outcome_prolonged_los_yn']

In [32]:
# Convert variables to categorical
X[binary_list] = X[binary_list].astype('category')
X[category_list] = X[category_list].astype('category')

In [33]:
# Drop features that have >= 99% missing values And features in drop list
#missing_pct = X.isnull().mean() * 100
#cols_to_drop = missing_pct[missing_pct >= 99].index.tolist()
#final_drop_list = list(set(drop_list + cols_to_drop))
#X = X.drop(columns=final_drop_list)

#X_1b = X_1b.reset_index(drop=True)
#y_1b = y_1b.reset_index(drop=True)

In [67]:
missing_pct = X.isnull().mean() * 100

AttributeError: 'list' object has no attribute 'drop'

In [71]:
pd.set_option('display.max_rows', 400)

In [73]:
continuous_list

['viz_age',
 'thro_door_to_admit_order_hrs',
 'thro_admit_order_to_first_bed_hrs',
 'thro_last_bed_to_ed_dept_hrs',
 'thro_admit_order_to_ed_dept_hrs',
 'summary_first_edd_to_discharge_hrs',
 'summary_first_edd_doc_to_discharge_hrs',
 'summary_last_edd_to_discharge_hrs',
 'summary_last_edd_doc_to_discharge_hrs',
 'summary_first_rfd_to_discharge_hrs',
 'summary_last_rfd_to_discharge_hrs',
 'summary_total_hrs_nmr',
 'summary_total_hrs_mrwb',
 'summary_total_hrs_mrnb',
 'rfd_first_nmr_to_discharge_hrs',
 'rfd_first_mrnb_to_discharge_hrs',
 'rfd_first_mrwb_to_discharge_hrs',
 'rfd_last_nmr_to_discharge_hrs',
 'rfd_last_mrnb_to_discharge_hrs',
 'rfd_last_mrwb_to_discharge_hrs',
 'con_max_consult_order_to_sign_any_hrs',
 'con_avg_consult_order_to_sign_any_hrs',
 'con_max_consult_note_to_sign_any_hrs',
 'con_avg_consult_note_to_sign_any_hrs',
 'con_max_consult_order_to_sign_colon_and_rectal_hrs',
 'con_max_consult_order_to_sign_diabetes_hrs',
 'con_max_consult_order_to_sign_hematology_hrs',
 

In [72]:
missing_pct[missing_pct != 0].reset_index()

,index,0
0,viz_primary_payer_name,3.786787
1,thro_door_to_admit_order_hrs,6.194690
2,thro_admit_order_to_first_bed_hrs,11.586746
3,thro_last_bed_to_ed_dept_hrs,11.566166
4,thro_admit_order_to_ed_dept_hrs,6.194690
5,thro_boarding_yn,6.194690
6,thro_ed_arrival_time,5.844824
7,thro_ed_arrival_day,5.844824
8,thro_admit_or_obs_order_time,6.194690
9,thro_admit_or_obs_order_day,6.194690


In [61]:
#these are patients who never had an order, so will impute with 0
category_list

['viz_primary_payer_name',
 'viz_disp_collapsed',
 'viz_ethnicity',
 'viz_race',
 'viz_race_collapsed',
 'viz_language',
 'viz_insurance',
 'viz_service_collapsed',
 'viz_ynhhs_sg2_service',
 'viz_drg',
 'viz_admission_day',
 'viz_discharged_day',
 'thro_ed_arrival_time',
 'thro_ed_arrival_day',
 'thro_admit_or_obs_order_time',
 'thro_admit_or_obs_order_day',
 'thro_first_bed_assigned_time',
 'thro_first_bed_assigned_day',
 'thro_last_bed_assigned_time',
 'thro_last_bed_assigned_day',
 'thro_ed_departure_time',
 'thro_ed_departure_day',
 'summary_pt_consult_order_time',
 'summary_pt_consult_order_day',
 'summary_sw_consult_order_time',
 'summary_sw_consult_order_day',
 'summary_first_edd_time',
 'summary_first_edd_day',
 'summary_first_edd_doc_time',
 'summary_first_edd_doc_day',
 'summary_last_edd_time',
 'summary_last_edd_day',
 'summary_last_edd_doc_time',
 'summary_last_edd_doc_day',
 'summary_first_rfd_status',
 'summary_first_rfd_time',
 'summary_first_rfd_day',
 'summary_last_rf

In [44]:
X = X.drop(columns=drop_list).reset_index(drop=True)

In [47]:
X.shape

(4859, 332)

In [49]:
#todo: this generates an error
#gss = GroupShuffleSplit(n_splits=2, test_size=0.2, random_state=42)
#train_ix, val_ix = next(gss.split(X, y, groups=X['pat_mrn_id']))
#X_train = X.loc[train_ix]
#y_train = y.loc[train_ix]
#X_val = X.loc[val_ix]
#y_val = y.loc[val_ix]
#X_train = X_train.drop(['pat_mrn_id'], axis=1).set_index('pat_enc_csn_id')
#X_val = X_val.drop(['pat_mrn_id'], axis=1).set_index('pat_enc_csn_id')

In [54]:
X_main, X_test, y_main, y_test = train_test_split(X, y,  test_size=0.15, random_state=42)
X_train, y_train, X_val, y_val = train_test_split(X_main,y_main,test_size=0.15, random_state=42)

In [55]:
X_train.shape

(3510, 332)

In [77]:
# Feature preprocessing pipeline
feature_preprocess_pipeline = Pipeline(steps=[

        # Missing value indicator for categoricals and continuous variables
        ('missing_indicator', AddMissingIndicator(variables=category_list + continuous_list)),

        # Missing value imputation
            # Impute missing values with 0 for discrete variables
            ('arbitrary_number_imputer', ArbitraryNumberImputer(arbitrary_number = 0, variables = discrete_list+continuous_list)),

            # Impute missing values with median for continuous variables 
#            ('median_imputer', MeanMedianImputer(imputation_method='median', variables = continuous_list)),

            # Impute missing values with adding 'missing' category for categorical variables 
            ('categorical_imputer', CategoricalImputer(variables = category_list)),

            # Impute missing values with 0 for binary variables 
            ('binary_imputer', CategoricalImputer(fill_value='0', variables = binary_list)),
       
    
        # Rare encoding for categorical variables
        ('rare_label_encoder', RareLabelEncoder(tol=0.01, n_categories=5, max_n_categories=10, variables=category_list)),
    
        # OneHotEncoding for categoricals
        ('one_hot_category', OneHotEncoder(variables=category_list)),

        # Outlier handling & normalization for continous variables
        ('outlier_handling', Winsorizer(capping_method='gaussian', tail='right', fold=3, variables=continuous_list)),

        ('normalize', SklearnTransformerWrapper(MinMaxScaler(feature_range=(0, 1)), variables=continuous_list)),

        # discretize for discrete variables
        ('discretize', DecisionTreeDiscretiser(cv=3, scoring='roc_auc', regression=False, random_state=42, variables=discrete_list))
    
])

In [78]:
# Apply the pipeline
X_train_preprocessed = feature_preprocess_pipeline.fit_transform(X_train, y_train)
X_val_preprocessed = feature_preprocess_pipeline.transform(X_val)
X_test_preprocessed = feature_preprocess_pipeline.transform(X_test)

/gpfs/milgram/project/rtaylor/imc33/conda_envs/aitc/lib/python3.12/site-packages/feature_engine/imputation/missing_indicator.py:162: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[indicator_names] = X[self.variables_].isna().astype(int)
/gpfs/milgram/project/rtaylor/imc33/conda_envs/aitc/lib/python3.12/site-packages/feature_engine/imputation/missing_indicator.py:162: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[indicator_names] = X[self.variables_].isna().astype(int)
/gpfs/milgram/project/rtaylor/imc33/conda_envs/aitc/lib/py

ValueError: Some of the variables to transform contain inf values. Check and remove those before using this transformer.